### Главы 11 - 12

In [2]:
import sys

# скачиваем отзывы
raw_reviews = []
with open('reviews.txt') as f:
    for raw in f.readlines():
        raw_reviews.append(raw)

# скачиваем метки
raw_labels = []
with open('labels.txt') as f:
    for label in f.readlines():
        raw_labels.append(label)

print(raw_reviews[:2])
print(raw_labels[:2])

['bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \n', 'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience 

In [3]:
# собираем токены в список множеств
tokens = list(map(lambda x: set(x.split(" ")), raw_reviews))
print(f'len(tokens):{len(tokens)}\ntype(tokens) {type(tokens)}\ntype(tokens[0]) {type(tokens[0])}')

len(tokens):25000
type(tokens) <class 'list'>
type(tokens[0]) <class 'set'>


In [4]:
# составляем словарь, очищаем от пустых строк
vocab = set()
for sent in tokens:
    for word in sent:
        if len(word)>0:
            vocab.add(word)
# делаем из множества список
vocab = list(vocab)
len(vocab)

74074

In [5]:
# пронумеруем каждое слово и поместим в словарь
word2index = {}
for n,w in enumerate(vocab):
    word2index[w] = n

len(word2index.keys())

74074

In [6]:
# формируем массив для модели (предикторы)
input_dataset = []
for sent in tokens:
    sent_indices = []
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            continue
    input_dataset.append(list(set(sent_indices)))
# получили список из списков индексов для каждого отзыва
len(input_dataset)

25000

In [7]:
# формируем массив для модели (целевая переменная)
target_dataset = []
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

# получили список меток (1 - положительный, 0 - отрицательный)
len(target_dataset)

25000

Векторное представление

In [8]:
# разберемся с векторным представлением
import numpy as np
aa = np.arange(1,16).reshape(3,5)
# матрица весов
aa

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15]])

In [9]:
bb = np.ones(3, dtype=int).reshape(-1,3)
bb[:,1] = 0
# один вектор наблюдений из нулей и единиц
bb

array([[1, 0, 1]])

In [10]:
# скалярное произведение это сумма по столбцам без учета строк 
# номера которых равны индексам нулей в векторе наблюдений
bb.dot(aa)

array([[12, 14, 16, 18, 20]])

In [11]:
np.sum(aa[[0,2],:], axis=0)

array([12, 14, 16, 18, 20])

In [12]:
# То есть не надо делать дорогие матричные перемножения, достаточно сделать сложение! Особенно когда вектор длиной 74074.
# отразим это векторное представление в коде (для входного слоя)

In [13]:
import numpy as np
np.random.seed(1)

sigmoid = lambda x: 1/(1 + np.exp(-x))
alpha, iterations, hidden_size = 0.01, 3, 100

weights_0_1 = 0.2 * np.random.random((len(vocab), hidden_size)) - 0.1 # (74074,100)
weights_1_2 = 0.2 * np.random.random((hidden_size, 1)) - 0.1 # (100,1)

correct, total = 0, 0

for iteration in range(iterations):
    for i in range(len(input_dataset) - 1000): # для обучения берем первые 24 000 примеров
        x, y = (input_dataset[i], target_dataset[i])
        
        layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0).reshape(1,-1)) # (1,100)
        layer_2 = sigmoid(layer_1.dot(weights_1_2)) # (1,100)dot(100,1)=(1,1)
        
        layer_2_delta = layer_2 - y # (1,1)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) # (1,1)dot(1,100)=(1,100)
        
        weights_1_2 -= layer_1.T * layer_2_delta * alpha
        weights_0_1[x] -= layer_1_delta * alpha
        
        if np.abs(layer_2_delta) < 0.4: # если абсолютная ошибка меньше 0,5, то засчитываем как правильный ответ
            correct += 1
        total += 1
        
        if i % 3000 == 0 or i == (len(input_dataset) - 1000 - 1):
            progress = 100*i/len(input_dataset)
            print(f"Iter: {iteration} Progress: {round(progress, 2)} % Training Acc: {round(100*correct/total, 3)}%")
    print()

# проверка на тесте
correct, total = 0, 0
for i in range(len(input_dataset)-1000, len(input_dataset)):
    
    x, y = input_dataset[i], target_dataset[i]
    layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0)) # (1,100)
    layer_2 = sigmoid(layer_1.dot(weights_1_2)) # (1,100)dot(100,1)=(1,1)
    if np.abs(layer_2 - y) < 0.4: # если абсолютная ошибка меньше 0,5, то засчитываем как правильный ответ
            correct += 1
    total += 1
print(f'Test Acc: {round(100*correct/total, 5)}%', '\n', '-'*30)            

Iter: 0 Progress: 0.0 % Training Acc: 0.0%
Iter: 0 Progress: 12.0 % Training Acc: 40.453%
Iter: 0 Progress: 24.0 % Training Acc: 57.057%
Iter: 0 Progress: 36.0 % Training Acc: 64.637%
Iter: 0 Progress: 48.0 % Training Acc: 68.728%
Iter: 0 Progress: 60.0 % Training Acc: 71.089%
Iter: 0 Progress: 72.0 % Training Acc: 72.468%
Iter: 0 Progress: 84.0 % Training Acc: 73.973%
Iter: 0 Progress: 96.0 % Training Acc: 75.129%

Iter: 1 Progress: 0.0 % Training Acc: 75.13%
Iter: 1 Progress: 12.0 % Training Acc: 76.345%
Iter: 1 Progress: 24.0 % Training Acc: 77.264%
Iter: 1 Progress: 36.0 % Training Acc: 78.119%
Iter: 1 Progress: 48.0 % Training Acc: 78.892%
Iter: 1 Progress: 60.0 % Training Acc: 79.506%
Iter: 1 Progress: 72.0 % Training Acc: 79.993%
Iter: 1 Progress: 84.0 % Training Acc: 80.532%
Iter: 1 Progress: 96.0 % Training Acc: 80.95%

Iter: 2 Progress: 0.0 % Training Acc: 80.95%
Iter: 2 Progress: 12.0 % Training Acc: 81.438%
Iter: 2 Progress: 24.0 % Training Acc: 81.784%
Iter: 2 Progress: 36

#### Сравнение векторных представлений (весов) слов

Нужно выбрать слово, посмотреть на его вектор весов во входном слое и поискать близкие векторы.
Сравнимаем строки в матрице весов!

In [14]:
from collections import Counter
# import math

def similar(target='beautiful'):
    target_index = word2index[target] # индекс слова в мешке слов
    scores = Counter() # словарь для учета частот слов
    for word, index in word2index.items(): # бежим по ключам, значениям мешка слов
        raw_difference = weights_0_1[index] - weights_0_1[target_index]
        scores[word] = -1 * np.sqrt(np.sum(raw_difference**2)) # добавляем с минусом евклидово расстояние
        
    return scores.most_common(10)

In [15]:
for w in ['beautiful', 'terrible']:
    print(similar(target=w), '\n')

[('beautiful', -0.0), ('tight', -0.7209523658054507), ('realistic', -0.7236111385216073), ('flawless', -0.726916481712693), ('notch', -0.7335271507538228), ('negative', -0.7411976375234283), ('awesome', -0.7428778479027082), ('treat', -0.7455946929079215), ('worlds', -0.7478677576454998), ('impressed', -0.7504621838874066)] 

[('terrible', -0.0), ('fails', -0.756374397231069), ('avoid', -0.7991837573351354), ('wooden', -0.8007299958561985), ('boring', -0.8087608383893226), ('badly', -0.815500442396233), ('redeeming', -0.8338771177322658), ('horrible', -0.8388270767871248), ('annoying', -0.8405398955631521), ('forgettable', -0.8432361209217222)] 



In [16]:
similar(target='beautiful')

[('beautiful', -0.0),
 ('tight', -0.7209523658054507),
 ('realistic', -0.7236111385216073),
 ('flawless', -0.726916481712693),
 ('notch', -0.7335271507538228),
 ('negative', -0.7411976375234283),
 ('awesome', -0.7428778479027082),
 ('treat', -0.7455946929079215),
 ('worlds', -0.7478677576454998),
 ('impressed', -0.7504621838874066)]

In [17]:
similar(target='terrible')

[('terrible', -0.0),
 ('fails', -0.756374397231069),
 ('avoid', -0.7991837573351354),
 ('wooden', -0.8007299958561985),
 ('boring', -0.8087608383893226),
 ('badly', -0.815500442396233),
 ('redeeming', -0.8338771177322658),
 ('horrible', -0.8388270767871248),
 ('annoying', -0.8405398955631521),
 ('forgettable', -0.8432361209217222)]

In [18]:
similar(target='atmosphere')

[('atmosphere', -0.0),
 ('powerful', -0.691491458151142),
 ('sweet', -0.7029953285676174),
 ('outstanding', -0.7084380957648823),
 ('beautifully', -0.7118115346774474),
 ('liked', -0.7122365707933771),
 ('tight', -0.729463536913003),
 ('episodes', -0.7376865613748727),
 ('criticism', -0.741091737358788),
 ('spectacular', -0.7415743885841557)]

In [19]:
# Сеть группирует слова через призму "положительный или отрицательный отзыв". 
# Смысл нейрона определяется предсказываемыми метками

In [20]:
import sys, random, math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)

tokens = list(map(lambda x: (x.split(" ")), raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0], wordcnt.most_common())))

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
    
concatenated = []
input_dataset = []

for sent in tokens:
    sent_indices = []
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            continue
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)

random.shuffle(input_dataset) # Берет последовательность и возвращает ее в перемешанном состоянии
alpha, iterations, hidden_size, window, negative = 0.05, 2, 50, 2, 5

weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab), hidden_size) * 0

layer_2_target = np.zeros(negative + 1)
layer_2_target[0] = 1

def similar(target='beautiful'):
    target_index = word2index[target] # индекс слова в мешке слов
    scores = Counter() # словарь для учета частот слов
    for word, index in word2index.items(): # бежим по ключам, значениям мешка слов
        raw_difference = weights_0_1[index] - weights_0_1[target_index]
        scores[word] = -1 * np.sqrt(np.sum(raw_difference**2)) # добавляем с минусом евклидово расстояние
        
    return scores.most_common(10)

sigmoid = lambda x: 1/(1 + np.exp(-x))

for rev_i, review in enumerate(input_dataset * iterations):
    for target_i in range(len(review)):
        
        target_samples = [review[target_i]] + list(concatenated\
                                                   [(np.random.rand(negative) * len(concatenated)).astype('int').tolist()])
        left_context = review[max(0, target_i - window):target_i]
        right_context = review[target_i + 1 : min(len(review), target_i + window)]
        
        layer_1 = np.mean(weights_0_1[left_context + right_context], axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])
        
        weights_0_1[left_context + right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha
        
    if rev_i % 2500 == 0 :
        progress = 100*rev_i/(len(input_dataset)*iterations)
        print(f"Progress: {round(progress, 2)}% {similar('terrible')}")
print(similar('terrible'))

Progress: 0.0% [('terrible', -0.0), ('weened', -0.3684632319655597), ('dullllllllllll', -0.38338063344860374), ('memories', -0.3848793731367956), ('preaches', -0.38579367148753596), ('deadhead', -0.3913601228910714), ('janet', -0.3915047699956218), ('flamethrowers', -0.3947111977040416), ('jermy', -0.39762091865648075), ('exceptionally', -0.397740171574861)]
Progress: 5.0% [('terrible', -0.0), ('fantastic', -1.8009723152804034), ('perfectly', -1.8084681166926186), ('ridiculous', -1.810893990647885), ('horrible', -1.8534342521428562), ('essentially', -1.8558070053929663), ('solid', -1.9858713078140156), ('okay', -1.9984142687261657), ('cute', -2.004782985413128), ('carrying', -2.011581502113405)]
Progress: 10.0% [('terrible', -0.0), ('brilliant', -2.0549438356875007), ('fantastic', -2.055777101022888), ('unique', -2.067346433621027), ('magnificent', -2.166955632968605), ('manner', -2.1923149174754), ('l', -2.1990597551875735), ('student', -2.2130714789228394), ('teenager', -2.2176128086

In [42]:
np.save('weights_part_4', [weights_0_1, weights_1_2])
# np.save('weights_part_4', weights_1_2)

In [44]:
np.load('weights_part_4.npy').shape

(2, 74075, 50)

Словесная аналогия

In [21]:
def analogy(positive=['terrible', 'good'], negative=['bad']):
    
    norms = np.sum(weights_0_1 * weights_0_1, axis = 1)
    norms.resize(norms.shape[0], 1)
    
    normed_weights = weights_0_1 * norms
    
    query_vect = np.zeros(len(weights_0_1[0]))
    
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
        
    scores = Counter()
    
    for word, index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference ** 2
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [22]:
# 'terrible' - 'bad' + 'good'
analogy(['terrible', 'good'], ['bad'])

[('superb', -221.46820338343645),
 ('terrific', -221.8148593701283),
 ('decent', -221.90054276067337),
 ('fine', -221.95167472296896),
 ('perfect', -222.19214848996603),
 ('nice', -222.28298362559514),
 ('great', -222.3216032822223),
 ('worth', -222.3327471074601),
 ('brilliant', -222.35552439378316)]

In [23]:
# 'elizabeth' - 'she' + 'he'
analogy(['elizabeth', 'he'], ['she'])

[('christopher', -198.31092886508438),
 ('mr', -198.9767936617917),
 ('john', -199.2148675518587),
 ('de', -199.23572493921867),
 ('it', -199.25465723045528),
 ('william', -199.2559490448937),
 ('him', -199.29416405936328),
 ('tom', -199.35824539168513),
 ('david', -199.3807888925693)]

Усредняем вектора слов и предложений (глава 12)

In [334]:
import numpy as np
norms = np.sum(weights_0_1 ** 2, axis=1) # суммируем по горизонтали по каждой строке, т.е. 
                                        # для каждого слова его веса возводим в квадрат (делаем положительными)
                                        # и суммируем
norms = norms.reshape(-1,1) # добавляем измерение ширины 1
normed_weights = weights_0_1 * norms # нормируем веса, домножив на положительное число

def make_sent_vect(words):
    # из словаря word2index, в котором каждому слову (ключ) присвоен номер (значение),
    # отбираем индексы тех слова, которые есть у нас (переданы в функцию)
    # и помещаем в список indices
    indices = list(map(lambda x: word2index[x], filter(lambda x: x in word2index, words)))
    # возвращаем усредненный индекс для нашего набора слов (усредняем по столбцам)
    return np.mean(normed_weights[indices], axis=0)

reviews2vectors = [] # пустой список для усредненных векторов
for review in tokens: # берем список из отдельных слов, относящийся к одному отзыву
    reviews2vectors.append(make_sent_vect(review)) # наполняем список
reviews2vectors = np.array(reviews2vectors) # переводим в np массив (25000 отзывов, 50 значений усредненного вектора весов)

def most_similar_reviews(review):
    v = make_sent_vect(review) # (1,50)
    scores = Counter()
    for i, val in enumerate(reviews2vectors.dot(v.T)): # (25000,50) dot (50,1) = (25000,1) т.е. 25 тыс. расстояний
        scores[i] = val # записываем в словарь для каждого отзыва вектор-столбец из 25 тыс. расстояний
        
        most_similar = []
        
        for idx, score in scores.most_common(5): # most_common возвращает кортежи
            most_similar.append(raw_reviews[idx][:100]) # из "сырого" списка отзывов по индексу достаем отзыв 
                                                        # и сохраняем первые 40 символов 
    return most_similar

most_similar_reviews(['boring', 'awful'])

C:\Users\lpikulev\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\lpikulev\Anaconda3\lib\site-packages\numpy\core\_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


['homelessness  or houselessness as george carlin stated  has been an issue for years but never a plan',
 'brilliant over  acting by lesley ann warren . best dramatic hobo lady i have ever seen  and love sce',
 'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life',
 'airport    starts as a brand new luxury    plane is loaded up with valuable paintings  such belongin',
 'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terr']

Сейчас наш алгоритм никак не учитывает порядок слов. Для того, чтобы результат зависел от порядка слов, поэкспериментируем с перемножением векторов слов на единичные матрицы (частный случай матриц перехода).

In [335]:
import numpy as np

a = np.array([1,2,3])
b = np.array([0.1,0.2,0.3])
c = np.array([-1,-0.5,0])
d = np.array([0,0,0])

identity = np.eye(3) # создает единичные матрицы
print(identity)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [336]:
for v in [a,b,c,d]:
    print(v.dot(identity))

[1. 2. 3.]
[0.1 0.2 0.3]
[-1.  -0.5  0. ]
[0. 0. 0.]


In [337]:
this = np.array([2,4,6])
movie = np.array([10,10,10])
rocks = np.array([1,1,1])

print(this + movie + rocks) # просто складываем векторы слов
print((this.dot(identity) + movie).dot(identity) + rocks) # между сложением векторов слов умножаем промежуточные суммы на ед. матрицу

[13 15 17]
[13. 15. 17.]


Прямое распространение на Python

In [338]:
import numpy as np

def softmax(x_):
    x = np.atleast_2d(x_) # atleast_2d сразу решейпит в (1, -1) !
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True) # keepdims=True также решейпит в (1, -1) !

# векторные представления слов
word_vects = {}
word_vects['yankees'] = np.array([[0., 0., 0.]])
word_vects['bears'] = np.array([[0., 0., 0.]])
word_vects['braves'] = np.array([[0., 0., 0.]])
word_vects['red'] = np.array([[0., 0., 0.]])
word_vects['sox'] = np.array([[0., 0., 0.]])
word_vects['lose'] = np.array([[0., 0., 0.]])
word_vects['defeat'] = np.array([[0., 0., 0.]])
word_vects['beat'] = np.array([[0., 0., 0.]])
word_vects['tie'] = np.array([[0., 0., 0.]])

# веса для прогнозирования следующее слово по матрице предложения из 3 слов 
sent2output = np.random.rand(3, len(word_vects))
# матрица перехода (вначале единичная)
identity = np.eye(3)

Прямое распространение

In [367]:
# создаем предложение в векторном представлении
layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['sox']
layer_2 = layer_1.dot(identity) + word_vects['defeat']
# делаем прогнозы
pred = softmax(layer_2.dot(sent2output))
print(pred)

[[1. 1. 1. 1. 1. 1. 1. 1. 1.]]


Обратное распространение

In [368]:
# закодируем правильный ответ (предсказание слова 'yankees')
y = np.array([1,0,0,0,0,0,0,0,0])
alpha = 0.01

## спускаемся вниз
pred_delta = pred - y
layer_2_delta = pred_delta.dot(sent2output.T)

defeat_delta = layer_2_delta * 1 # получаем дельту для обучения матрицы (лучше отображать слово)
layer_1_delta = layer_2_delta.dot(identity.T) # получаем дельту для обучения матрицы перехода (рекуррентной матрицы)

sox_delta = layer_1_delta * 1
layer_0_delta = layer_1_delta.dot(identity.T)

## поднимаемся вверх

# обучаем сами вектора слов
word_vects['red'] -= layer_0_delta * alpha
word_vects['sox'] -= sox_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha

# обучаем матрицу перехода от 1 слоя к 2 и от 2 к 3
identity -= np.outer(layer_0, layer_1_delta) # np.outer перемножает между собой элементы в одинаковых позициях
identity -= np.outer(layer_1, layer_2_delta)

# обучаем матрицу весов
sent2output -= np.outer(layer_2, pred_delta) * alpha

In [369]:
# обучим на датасете Babi
import re
regex = re.compile('[0-9?!.,\t\n]+')

tokens = []
with open('qa1_single-supporting-fact_train.txt', 'r') as f:
    for raw in f.readlines()[:1000]:
        tokens.append(regex.sub('', raw).split(" ")[1:])
#         tokens.append(regex.sub('', raw.lower()).split(" ")[1:])

In [370]:
tokens[:3]

[['Mary', 'moved', 'to', 'the', 'bathroom'],
 ['John', 'went', 'to', 'the', 'hallway'],
 ['Where', 'is', 'Mary', 'bathroom']]

In [371]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)
vocab = list(vocab)

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i
    
def words2indices(sentence):
    idx = []
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x)) # нормируем вектор (это не обязательно)
    return e_x / e_x.sum(axis=0) # выдаем вектор, сумма элементов которого равна 1

In [372]:
np.random.seed(1)
emded_size = 10 # величина векторного представления (сколькими значениями кодируем отзыв)

embed = (np.random.rand(len(vocab), emded_size) - 0.5) * 0.1 # матрица входных весов (19,10)
recurrent = np.eye(emded_size) # рекуррентная матрица (вначале единичная, потом обучится)
start = np.zeros(emded_size) # представление для пустого предложения
decoder = (np.random.rand(emded_size, len(vocab)) - 0.5) * 0.1 # матрица выходных весов
one_hot = np.eye(len(vocab)) # матрица поиска выходных весов (там 70 тыс строк и в каждой строке по единице
                            # в позиции, соответствующей индексу)

In [373]:
tokens[0]

['Mary', 'moved', 'to', 'the', 'bathroom']

In [374]:
len(vocab)

19

In [375]:
word2index

{'went': 0,
 'Mary': 1,
 'John': 2,
 'back': 3,
 'to': 4,
 'bathroom': 5,
 'journeyed': 6,
 'garden': 7,
 'Daniel': 8,
 'moved': 9,
 'Sandra': 10,
 'office': 11,
 'Where': 12,
 'kitchen': 13,
 'bedroom': 14,
 'the': 15,
 'hallway': 16,
 'is': 17,
 'travelled': 18}

In [376]:
vocab

['went',
 'Mary',
 'John',
 'back',
 'to',
 'bathroom',
 'journeyed',
 'garden',
 'Daniel',
 'moved',
 'Sandra',
 'office',
 'Where',
 'kitchen',
 'bedroom',
 'the',
 'hallway',
 'is',
 'travelled']

In [377]:
def predict(sent):
    layers = [] # список слоев (коих м.б. разное количество в зависимости от числа слов в предложении)
    layer = {} # пока пустой словарь
    layer['pred'] = np.zeros((1,len(vocab))) # только для демонстрации записываем в словарь пустое прдсказание, далее
                                            # будем обновлять
    layer['hidden'] = start # в словарь записываем скрытый слой, на который передается вектор из нулей длиной embed_size
                            # дальше будем обновлять значения по этому ключу
    layers.append(layer) # словарь добавляем в список
    
    loss = 0 # начальное значение ошибки (ф-я потерь)
        
    for target_i in range(len(sent)): # для каждого слова из предложения
        layer = {} # создаем новый пустой словарь
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder)) # записываем в него попытку предсказания:
                                                                    # первым выбирается start - вектор из нулей 
                                                                    # по количеству слов в предложении (1,10),
                                                                    # т.е. берем последний записанный словарь из списка layers
                                                                    # и в нем по ключу 'hidden'
                                                                    # находим вектор предложения 
                                                                    # и скалярно умножаем на матрицу выходных весов (10,19)
                                                                    # получаем вектор (1,19)
#         print(layer['pred'].shape)
#         print(layer['pred'][target_i])
        loss += -np.log(layer['pred'][sent[target_i]]) # учитываем лосс: десятичный логарифм от 0 (если ошибка) и от 1 (если верно)
#         print(loss)
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[target_i] # обновляем значения нейронов скрытого слоя^
                    # в первую итерацию вместо вектора из 10 нулей получаем вектор из 9 нулей и одной единицы
                                                            
        layers.append(layer) # список layers будет накапливать элементы, но использовать мы будем их с последнего
    
    return layers, loss

Обратное распространение

In [378]:
for iteration in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iteration% len(tokens)][1:]) # берем очередной отзыв
    layers, loss = predict(sent) # получаем список значений на скрытом слое и прогноз
    
    for layer_idx in reversed(range(len(layers))): # [4, 3, 2, 1, 0]
        layer = layers[layer_idx] # берем слой (с последнего)
        target = sent[layer_idx - 1]
        
        if layer_idx > 0: # если не первый слой
            layer['output_delta'] = layer['pred'] - one_hot[target] # сравниваем предсказание с правдой
            new_hidden_delta = layer['output_delta'].dot(decoder.T)
            
            if layer_idx == len(layers) - 1 :
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + \
                layers[layer_idx+1]['hidden_delta'].dot(recurrent.T)
        else: # если первый слой
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta']\
            .dot(recurrent.T)

In [379]:
loss

11.787363136429493

ПОЯСНЕНИЕ ЗАПУТАННОГО КОДА ИЗ КНИГИ:

In [380]:
tokens[0]

['Mary', 'moved', 'to', 'the', 'bathroom']

In [381]:
tokens2 = [tokens[0]]

for iteration in range(1):
    alpha = 0.001
    sent = words2indices(tokens2[iteration% len(tokens2)][1:]) # берем очередной отзыв
    print(f'ИТЕРАЦИЯ №{iteration}, вектор предложения: {sent} (без первого элемента)')
    layers, loss = predict(sent) # получаем список значений на скрытом слое и прогноз
    print('Количество слоев (len(layers)):', len(layers))
    print('Ошибка (loss)', loss, '\n')
    
    for layer_idx in reversed(range(len(layers))): # [4, 3, 2, 1, 0]
        print('Работаем со слоем', layer_idx)
        layer = layers[layer_idx] # берем слой (с последнего)
        print(f'Извлекаем из списка слоев слой №{layer_idx}')
        target = sent[layer_idx - 1]
        print(f'Целевое слово ("{vocab[target]}") под индексом: {target}')
        
        if layer_idx > 0: # если не первый слой
            print('\tЭто не самый первый слой!')
            layer['output_delta'] = layer['pred'] - one_hot[target] # сравниваем предсказание с правдой
            print(f"\tСравнили предсказание (layer['pred']) с правдой (one_hot[target]), \
получили дельту (layer['output_delta']): \n{layer['output_delta']}")
            print('\tТеперь рапространим градиент (полученную дельту) на скрытый слой.')
            new_hidden_delta = layer['output_delta'].dot(decoder.T)
            print(f"\tПолучили new_hidden_delta", new_hidden_delta, '\n')
            print('\tТеперь проверим, с последним слоем мы вообще работаем или нет.')
            
            if layer_idx == len(layers) - 1 :
                print("\tЭто последний слой! Значит просто запоминаем для него полученный только что градиент.\n")
                layer['hidden_delta'] = new_hidden_delta
                print(f"\tПолучили: {layer['hidden_delta']}")
            else:
                print("\tЭто не последний слой! Значит запоминаем для него \
полученный только что градиент с добавлением градиента вышестоящего слоя.\n")
                layer['hidden_delta'] = new_hidden_delta + \
                layers[layer_idx+1]['hidden_delta'].dot(recurrent.T)
                print(f"\tПолучили: {layer['hidden_delta']}")
        else: # если первый слой
            print('\tЭто самый первый слой! Значит запоминаем для него полученный для слоя выше градиент \
(но не забываем распределить его умножением на матрицу current).')
            print(f"\tВот эта дельта берется (такая же, как слоем выше): {layers[layer_idx + 1]['hidden_delta']}\n")
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta']\
            .dot(recurrent.T)            

ИТЕРАЦИЯ №0, вектор предложения: [9, 4, 15, 5] (без первого элемента)
Количество слоев (len(layers)): 5
Ошибка (loss) 11.784163594563987 

Работаем со слоем 4
Извлекаем из списка слоев слой №4
Целевое слово ("bathroom") под индексом: 5
	Это не самый первый слой!
	Сравнили предсказание (layer['pred']) с правдой (one_hot[target]), получили дельту (layer['output_delta']): 
[ 0.05270767  0.05228045  0.05326576  0.05259298  0.0523218  -0.9473208
  0.0523481   0.05251092  0.05294389  0.05287013  0.05273489  0.05229335
  0.05270291  0.05250183  0.05254811  0.05249804  0.05264425  0.05240833
  0.05314741]
	Теперь рапространим градиент (полученную дельту) на скрытый слой.
	Получили new_hidden_delta [-0.03254191  0.03514132  0.02505568  0.02602693  0.02126818  0.00489023
  0.01235094 -0.00225488 -0.00374738 -0.04933733] 

	Теперь проверим, с последним слоем мы вообще работаем или нет.
	Это последний слой! Значит просто запоминаем для него полученный только что градиент.

	Получили: [-0.03254191 

Корректировка весов:

In [382]:
for iteration in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iteration% len(tokens)][1:]) # берем очередной отзыв
    layers, loss = predict(sent) # получаем список значений на скрытом слое и прогноз
#     print(loss)
    
    for layer_idx in reversed(range(len(layers))): # [4, 3, 2, 1, 0]
        layer = layers[layer_idx] # берем слой (с последнего)
        target = sent[layer_idx - 1]
        
        if layer_idx > 0: # если не первый слой
            layer['output_delta'] = layer['pred'] - one_hot[target] # сравниваем предсказание с правдой
            new_hidden_delta = layer['output_delta'].dot(decoder.T)
            
            if layer_idx == len(layers) - 1 :
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + \
                layers[layer_idx+1]['hidden_delta'].dot(recurrent.T)
        else: # если первый слой
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta']\
            .dot(recurrent.T)
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent)) # обновляем векторное представление предложения
    
    for layer_idx, layer in enumerate(layers[1:]):
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta'] * alpha / float(len(sent)))
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha/float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
    if iteration % 1000 == 0:
#         print(loss, float(len(sent)))
        print(f'Perplexity: {np.exp(loss/len(sent))}')

Perplexity: 19.030460861426217
Perplexity: 19.005338406741927
Perplexity: 18.983630600166556
Perplexity: 18.96162976790398
Perplexity: 18.93550027081478
Perplexity: 18.900274769526536
Perplexity: 18.848315746843465
Perplexity: 18.76630133653987
Perplexity: 18.628225780001614
Perplexity: 18.376310435967987
Perplexity: 17.85820485139596
Perplexity: 16.582238702769008
Perplexity: 13.796186842143726
Perplexity: 12.413079451432717
Perplexity: 11.652984093299983
Perplexity: 11.230679026774595
Perplexity: 10.993438271591858
Perplexity: 10.615148195524222
Perplexity: 9.94973814736961
Perplexity: 8.927886846032001
Perplexity: 7.786784807732955
Perplexity: 6.878345922615974
Perplexity: 6.323292210757849
Perplexity: 6.063338271167899
Perplexity: 6.015027161466452
Perplexity: 6.084479950256829
Perplexity: 6.192234499608798
Perplexity: 6.291687898302753
Perplexity: 6.339789845237176
Perplexity: 6.280486863188851


In [387]:
for iteration in range(50000):
    alpha = 0.001
    sent = words2indices(tokens[iteration% len(tokens)][1:]) # берем очередной отзыв
    layers, loss = predict(sent) # получаем список значений на скрытом слое и прогноз
#     print(loss)
    
    for layer_idx in reversed(range(len(layers))): # [4, 3, 2, 1, 0]
        layer = layers[layer_idx] # берем слой (с последнего)
        target = sent[layer_idx - 1]
        
        if layer_idx > 0: # если не первый слой
            layer['output_delta'] = layer['pred'] - one_hot[target] # сравниваем предсказание с правдой
            new_hidden_delta = layer['output_delta'].dot(decoder.T)
            
            if layer_idx == len(layers) - 1 :
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + \
                layers[layer_idx+1]['hidden_delta'].dot(recurrent.T)
        else: # если первый слой
            layer['hidden_delta'] = layers[layer_idx + 1]['hidden_delta']\
            .dot(recurrent.T)
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent)) # обновляем векторное представление предложения
    
    for layer_idx, layer in enumerate(layers[1:]):
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta'] * alpha / float(len(sent)))
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha/float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
#     if iteration % 1000 == 0:
#         print(loss, float(len(sent)))
#         print(f'Perplexity: {np.exp(loss/len(sent))}')

sent_index = 4
l, _ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])

for i, each_layer in enumerate(l[1:-1]):
    inp = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input: " + inp + (' ' * (12 - len(inp))) +\
         "True: " + true + (" " * (15 - len(true))) + "Pred: "+ pred)

['Sandra', 'moved', 'to', 'the', 'garden']
Prev Input: Sandra      True: moved          Pred: is
Prev Input: moved       True: to             Pred: to
Prev Input: to          True: the            Pred: the
Prev Input: the         True: garden         Pred: bedroom
